In [ ]:
"""
This module contains constants representing core & auxiliary fish body parts.
"""

UPPER_LIP = 'UPPER_LIP'
EYE = 'EYE'
PECTORAL_FIN = 'PECTORAL_FIN'
DORSAL_FIN = 'DORSAL_FIN'
PELVIC_FIN = 'PELVIC_FIN'
ADIPOSE_FIN = 'ADIPOSE_FIN'
ANAL_FIN = 'ANAL_FIN'
TAIL_NOTCH = 'TAIL_NOTCH'
UPPER_PRECAUDAL_PIT = 'UPPER_PRECAUDAL_PIT'
LOWER_PRECAUDAL_PIT = 'LOWER_PRECAUDAL_PIT'
HYPURAL_PLATE = 'HYPURAL_PLATE'

core_body_parts = sorted([UPPER_LIP,
                          EYE,
                          PECTORAL_FIN,
                          DORSAL_FIN,
                          PELVIC_FIN,
                          ADIPOSE_FIN,
                          ANAL_FIN,
                          TAIL_NOTCH])

auxiliary_body_parts = sorted([UPPER_PRECAUDAL_PIT,
                               LOWER_PRECAUDAL_PIT,
                               HYPURAL_PLATE])

all_body_parts = sorted(core_body_parts + auxiliary_body_parts)

In [ ]:
"""This module contains utility helper functions for the WeightEstimator class."""

from collections import namedtuple
from typing import Dict, List, Tuple
import numpy as np
import torch


CameraMetadata = namedtuple('CameraMetadata',
                            ['focal_length', 'focal_length_pixel', 'baseline_m',
                             'pixel_count_width', 'pixel_count_height', 'image_sensor_width',
                             'image_sensor_height'])


def get_left_right_keypoint_arrs(annotation: Dict[str, List[Dict]]) -> Tuple:
    """Gets numpy array of left and right keypoints given input keypoint annotation.
    Args:
        annotation: dict with keys 'leftCrop' and 'rightCrop'. Values are lists where each element
        is a dict with keys 'keypointType', 'xCrop' (num pixels from crop left edge),
        'yCrop' (num pixels from crop top edge), 'xFrame' (num pixels from full frame left edge),
        and 'yFrame' (num pixels from full frame top edge).
    Returns:
        X_left: numpy array containing left crop (xFrame, yFrame) for each key-point ordered
        alphabetically.
        X_right: same as above, but for right crop.
    """

    left_keypoints, right_keypoints = {}, {}
    for item in annotation['leftCrop']:
        body_part = item['keypointType']
        left_keypoints[body_part] = (item['xFrame'], item['yFrame'])

    for item in annotation['rightCrop']:
        body_part = item['keypointType']
        right_keypoints[body_part] = (item['xFrame'], item['yFrame'])

    left_keypoint_arr, right_keypoint_arr = [], []
    for body_part in core_body_parts:
        left_keypoint_arr.append(left_keypoints[body_part])
        right_keypoint_arr.append(right_keypoints[body_part])

    X_left = np.array(left_keypoint_arr)
    X_right = np.array(right_keypoint_arr)
    return X_left, X_right


def normalize_left_right_keypoint_arrs(X_left: np.ndarray, X_right: np.ndarray) -> Tuple:
    """Normalizes input left and right key-point arrays. The normalization involves (1) 2D
    translation of all keypoints such that they are centered, (2) rotation of the 2D coordiantes
    about the center such that the line passing through UPPER_LIP and fish center is horizontal.
    """

    # translate key-points, perform reflection if necessary
    upper_lip_idx = core_body_parts.index(UPPER_LIP)
    tail_notch_idx = core_body_parts.index(TAIL_NOTCH)
    if X_left[upper_lip_idx, 0] > X_left[tail_notch_idx, 0]:
        X_center = 0.5 * (np.max(X_left, axis=0) + np.min(X_left, axis=0))
        X_left_centered = X_left - X_center
        X_right_centered = X_right - X_center
    else:
        X_center = 0.5 * (np.max(X_right, axis=0) + np.min(X_right, axis=0))
        X_left_centered = X_right - X_center
        X_right_centered = X_left - X_center
        X_left_centered[:, 0] = -X_left_centered[:, 0]
        X_right_centered[:, 0] = -X_right_centered[:, 0]

    # rotate key-points
    upper_lip_x, upper_lip_y = tuple(X_left_centered[upper_lip_idx])
    theta = np.arctan(upper_lip_y / upper_lip_x)
    R = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ])

    D = X_left_centered - X_right_centered
    X_left_rot = np.dot(X_left_centered, R)
    X_right_rot = X_left_rot - D
    return X_left_rot, X_right_rot


def convert_to_world_point_arr(X_left: np.ndarray, X_right: np.ndarray,
                               camera_metadata: CameraMetadata) -> np.ndarray:
    """Converts input left and right normalized keypoint arrays into world coordinate array."""

    y_world = camera_metadata.focal_length_pixel * camera_metadata.baseline_m / \
              (X_left[:, 0] - X_right[:, 0])

    # Note: the lines commented out below are technically the correct formula for conversion
    # x_world = X_left[:, 0] * y_world / camera_metadata.focal_length_pixel
    # z_world = -X_left[:, 1] * y_world / camera_metadata.focal_length_pixel
    x_world = ((X_left[:, 0] * camera_metadata.image_sensor_width / camera_metadata.pixel_count_width) * y_world) / (camera_metadata.focal_length)
    z_world = (-(X_left[:, 1] * camera_metadata.image_sensor_height / camera_metadata.pixel_count_height) * y_world) / (camera_metadata.focal_length)
    X_world = np.vstack([x_world, y_world, z_world]).T
    return X_world


def stabilize_keypoints(X: np.ndarray) -> np.ndarray:
    """Transforms world coordinate array so that neural network inputs are stabilized"""
    X_new = np.zeros(X.shape)
    X_new[:, 0] = 0.5 * X[:, 0] / X[:, 1]
    X_new[:, 1] = 0.5 * X[:, 2] / X[:, 1]
    X_new[:, 2] = 0.05 / X[:, 1]
    return X_new


def convert_to_nn_input(annotation: Dict[str, List[Dict]], camera_metadata: CameraMetadata) \
        -> torch.Tensor:
    """Convrts input keypoint annotation and camera metadata into neural network tensor input."""
    X_left, X_right = get_left_right_keypoint_arrs(annotation)
    X_left_norm, X_right_norm = normalize_left_right_keypoint_arrs(X_left, X_right)
    X_world = convert_to_world_point_arr(X_left_norm, X_right_norm, camera_metadata)
    X = stabilize_keypoints(X_world)
    nn_input = torch.from_numpy(np.array([X])).float()
    return nn_input

In [ ]:
"""
This module contains the WeightEstimator class for estimating fish weight (g), length (mm), and
k-factor given input keypoint coordinates and camera metadata.
"""

from typing import Dict, Tuple
import torch
from torch import nn


class Network(nn.Module):
    """Network class defines neural-network architecture for both weight and k-factor estimation
    (currently both neural networks share identical architecture)."""

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(24, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        """Run inference on input keypoint tensor."""
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.output(x)
        return x


class WeightEstimator:
    """WeightEstimator class is used to predict fish weight, k-factor, and length
    given input keypoint annotations and camera metadata."""

    def __init__(self, weight_model_f: str, kf_model_f: str) -> None:
        """Initializes class with input weight and k-factor neural-networks."""
        self.weight_model = Network()
        self.weight_model.load_state_dict(torch.load(weight_model_f))
        self.weight_model.eval()

        self.kf_model = Network()
        self.kf_model.load_state_dict(torch.load(kf_model_f))
        self.kf_model.eval()

    @staticmethod
    def _get_model_input(annotation: Dict, camera_metadata: CameraMetadata) -> torch.Tensor:
        """Generates neural-network input tensor given annotation and camera_metadata."""
        X = convert_to_nn_input(annotation, camera_metadata)
        return X

    def predict_weight(self, annotation: Dict, camera_metadata: CameraMetadata) -> float:
        """Generates weight prediction given input annotation and camera metadata."""
        X = self._get_model_input(annotation, camera_metadata)
        weight = 1e4 * self.weight_model(X).item()
        return weight

    def predict_kf(self, annotation: Dict, camera_metadata: CameraMetadata) -> float:
        """Generates k-factor prediction gievn input annotation and camera metadata."""
        X = self._get_model_input(annotation, camera_metadata)
        kf = self.kf_model(X).item()
        return kf

    def predict(self, annotation: Dict, camera_metadata: CameraMetadata) -> Tuple:
        """Generates weight, k-factor, and length predictions given input annotation and camera
        metadata."""
        weight = self.predict_weight(annotation, camera_metadata)
        kf = self.predict_kf(annotation, camera_metadata)
        if weight * kf > 0:
            length = (1e5 * weight / kf) ** (1.0 / 3)
        else:
            length = 0
        return weight, length, kf

In [ ]:
import math
import json
import os
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta, time
from research.utils.data_access_utils import RDSAccessUtils
from research.weight_estimation.keypoint_utils.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
import matplotlib.pyplot as plt
from matplotlib.dates import AutoDateFormatter, AutoDateLocator

rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

In [ ]:
queryCache = {}

In [ ]:
gt = pd.read_csv('eide_langoy_singleweights.csv')

In [ ]:
plt.figure(figsize=(20, 10))

plt.hist(gt['weight'] * 1000 / 0.83, bins = 50)

In [ ]:
def get_angles(kp1, kp2):
    vector = kp1 - kp2
    x, y, z = vector / np.linalg.norm(vector)

    theta = math.atan(y / x) * np.sign(y)
    phi = math.acos(z)
    dtheta = math.degrees(theta)
    dphi = 90 - math.degrees(phi)
    
    return dtheta, dphi

# cm2 = CameraMetadata(
#     focal_length=4000 * 3.45e-6,
#     focal_length_pixel=4000,
#     baseline_m=0.1,
#     pixel_count_width=4000,
#     pixel_count_height=3096,
#     image_sensor_width=0.0141312,
#     image_sensor_height=0.01035
# )

weight_model_f = 'weight_model.pb'
kf_model_f = 'kf_model.pb'
weight_estimator = WeightEstimator(weight_model_f, kf_model_f)

def getPenDF(pen, akpd_filter):
    query = """
       SELECT * FROM prod.biomass_computations
        WHERE prod.biomass_computations.captured_at >= '%s'
        AND prod.biomass_computations.captured_at <= '%s'
        AND prod.biomass_computations.akpd_score > %0.4f
        AND prod.biomass_computations.pen_id = %i;
    """ % (pen['start_date'], pen['end_date'], akpd_filter, pen['pen_id'])

    if query in queryCache:
        df = queryCache[query].copy()
    else:
        df = rds_access_utils.extract_from_database(query)
        queryCache[query] = df.copy()

    df = df.sort_values('captured_at').copy(deep=True)
    df.index = pd.to_datetime(df.captured_at)
    dates = df.index.date.astype(str)
    df['date'] = dates
    df['hour'] = df.index.hour
    
    depths = []
    lengths = []
    widths = []
    lengths_adj = []
#     lengths_adj2 = []
    
    weights2 = []
    lengths2 = []
    kfs2 = []
    
    for idx, row in df.iterrows():
        ann, camera_metadata = row.annotation, row.camera_metadata
        wkps = pixel2world(ann['leftCrop'], ann['rightCrop'], camera_metadata)
        depth = np.median([wkp[1] for wkp in wkps.values()])
        vector = wkps['UPPER_LIP'] - wkps['TAIL_NOTCH']
        
        cm2 = CameraMetadata(
            focal_length=camera_metadata['focalLength'],
            focal_length_pixel=camera_metadata['focalLengthPixel'],
            baseline_m=camera_metadata['baseline'],
            pixel_count_width=camera_metadata['pixelCountWidth'],
            pixel_count_height=camera_metadata['pixelCountHeight'],
            image_sensor_width=camera_metadata['imageSensorWidth'],
            image_sensor_height=camera_metadata['imageSensorHeight']
        )
        
        weight, length, kf = weight_estimator.predict(row.annotation, cm2)
        
        weights2.append(weight)
        lengths2.append(length / 1000)
        kfs2.append(kf)
        
        centroid = .5 * (wkps['DORSAL_FIN'] + wkps['PELVIC_FIN'])
        angle = np.linalg.norm(np.array(get_angles(wkps['UPPER_LIP'], centroid)) - np.array(get_angles(centroid, wkps['TAIL_NOTCH'])))
        a = (wkps['UPPER_LIP'] - centroid) / np.linalg.norm(wkps['UPPER_LIP'] - centroid)
        b = (wkps['TAIL_NOTCH'] - centroid) / np.linalg.norm(wkps['TAIL_NOTCH'] - centroid)

        lengths_adj.append(np.linalg.norm(wkps['UPPER_LIP'] - centroid) + np.linalg.norm(centroid - wkps['TAIL_NOTCH']))
#         lengths_adj.append(np.linalg.norm((wkps['UPPER_LIP'] - wkps['TAIL_NOTCH']) * a))
#         lengths_adj2.append(np.linalg.norm((wkps['UPPER_LIP'] - wkps['TAIL_NOTCH']) * b))
        
        depths.append(depth)
        lengths.append(np.linalg.norm(vector))
        widths.append(np.linalg.norm(wkps['DORSAL_FIN'] - wkps['PELVIC_FIN']))
        
    df['depth'] = depths
    df['length'] = lengths
    df['width'] = widths
    df['length_adj'] = lengths_adj
#     df['length_adj2'] = lengths_adj2
    
    df['weight2'] = weights2
    df['length2'] = lengths2
    df['kf2'] = kfs2
    
    df['estimated_weight'] = get_weight(df.length)
    df['estimated_weight2'] = get_weight(df.length2)
    df['estimated_weight_adj'] = get_weight(df.length_adj)
#     df['estimated_weight'] = get_weight(df.length, df.width)
#     df['estimated_weight2'] = get_weight(df.length2, df.width)
#     df['estimated_weight_adj'] = get_weight(df.length_adj, df.width)
#     df['estimated_weight_adj2'] = get_weight(df.length_adj2, df.width2)
    
    
    return df

In [ ]:
pens = [
    {
        'pen_id': 108,
        'start_date': '2020-05-07',
        'end_date': '2020-05-10'
    },
    {
        'pen_id': 60,
        'start_date': '2020-08-22',
        'end_date': '2020-08-25'
    },
    {
        'pen_id': 145,
        'start_date': '2020-11-20',
        'end_date': '2020-11-23'
    },
    {
        'pen_id': 61,
        'start_date': '2019-11-30',
        'end_date': '2019-12-02'
    },
    {
        'pen_id': 153,
        'start_date': '2020-11-18',
        'end_date': '2020-11-21'
    }
]

akpd_score = 0.95

_df1 = getPenDF(pens[0], akpd_score)
_df2 = getPenDF(pens[1], akpd_score)
_df3 = getPenDF(pens[2], akpd_score)
_df4 = getPenDF(pens[3], akpd_score)
_df5 = getPenDF(pens[4], akpd_score)

df1 = _df1[(_df1.hour >= 6) & (_df1.hour <= 12)]
df2 = _df2[(_df2.hour >= 5) & (_df2.hour <= 15)]
df3 = _df3[(_df3.hour >= 7) & (_df3.hour <= 15)]
df4 = _df4[(_df4.hour >= 8) & (_df4.hour <= 14)]
df5 = _df5[(_df5.hour >= 7) & (_df5.hour <= 15)]

In [ ]:
df5[['estimated_weight_g', 'weight2']].head()

In [ ]:
np.mean(df5.estimated_weight_g), np.mean(df5.weight2)

mask = df5.estimated_weight_g > 8000
df6 = df5.copy()
df6.loc[mask, 'estimated_weight_g'] = df6[mask].weight2
np.mean(df6.estimated_weight_g), np.mean(df5.estimated_weight_g)

In [ ]:
row = df1.iloc[0]

cm = CameraMetadata(
    focal_length=4000 * 3.45e-6,
    focal_length_pixel=4000,
    baseline_m=0.1,
    pixel_count_width=4000,
    pixel_count_height=3096,
    image_sensor_width=0.0141312,
    image_sensor_height=0.01035
)

weight_model_f = 'weight_model.pb'
kf_model_f = 'kf_model.pb'
weight_estimator = WeightEstimator(weight_model_f, kf_model_f)
weight, length, kf = weight_estimator.predict(row.annotation, cm)

print(weight, length, kf)

In [ ]:
plt.hist(_df4.hour, bins = 24)

In [ ]:
# count, bins, _ = plt.hist(df3.estimated_weight_g, density = True, alpha = 0.5, color = 'red', bins = 20)
weight2 = get_weight(df3.length_adj)

count, bins, _ = plt.hist(weight2[weight2 < 12000], density = True, alpha = 0.5, color = 'blue', bins = 20)
# plt.hist(df3.estimated_weight_g, density = True, alpha = 0.5, color = 'red', bins = bins)


In [ ]:
df1.head()

In [ ]:
def get_weight(length):
    return (length * 23.6068) ** 3

# def get_weight(length, width):
#     return (length * 12.5471 + width * 46.1998) ** 3

# def get_weight(length, width):
#     return (length * 16.06328176 + width * 31.74291343) ** 3

# def get_weight(length, width):
#     return (length ** 3 * 13071.99583268 + width ** 3 * 7460.4478138)
 

In [ ]:
def get_weight3(length, factor):
    return ((length * 23.6068) ** 3) * factor

In [ ]:
np.mean(df1.estimated_weight_g), np.mean(gt.weight / .83 * 1000), np.mean(get_weight(df1.length_adj))

In [ ]:
np.mean(df2.estimated_weight_g), np.mean(pen5.weight / .83 * 1000), np.mean(get_weight(df2.length_adj))

In [ ]:
plt.hist(df1.estimated_weight_g, density = True, alpha = 0.5, color = 'red', bins = bins)

In [ ]:
count, bins, _ = plt.hist(gt['weight'] * 1000 / 0.83, density = True, color = 'blue', bins = 50)
plt.hist(df1.estimated_weight_g, density = True, alpha = 0.5, color = 'red', bins = bins)

In [ ]:
count, bins, _ = plt.hist(gt['weight'] * 1000 / 0.83, density = True, color = 'blue', bins = 50)
plt.hist(df1.weight2, density = True, alpha = 0.5, color = 'red', bins = bins)

In [ ]:
count, bins, _ = plt.hist(gt['weight'] * 1000 / 0.83, density = True, color = 'blue', bins = 50)
plt.hist(df1.estimated_weight2, density = True, alpha = 0.5, color = 'red', bins = bins)

In [ ]:
count, bins, _ = plt.hist(gt['weight'][gt.weight < (5 * .83)] * 1000 / 0.83, density = True, color = 'blue', bins = 50)
plt.hist(df1.estimated_weight_g[df1.estimated_weight_g < 5000], density = True, alpha = 0.5, color = 'red', bins = bins)

In [ ]:
a, b = np.mean(gt['weight'][gt.weight < (5 * .83)] * 1000 / 0.83), np.mean(df1.estimated_weight_g[df1.estimated_weight_g < 5000])

a, b, (a - b) / a

In [ ]:
count, bins, _ = plt.hist(gt['weight'] * 1000 / 0.83, density = True, color = 'blue', bins = 30)
plt.hist(get_weight(df1.length, df1.width), density = True, alpha = 0.5, color = 'red', bins = bins)

In [ ]:
plt.scatter(df1.estimated_weight_g, df1.weight2)
plt.plot(df1.estimated_weight_g, df1.estimated_weight_g, color = 'red')

In [ ]:
plt.scatter(df2.estimated_weight_g, df2.weight2)
plt.plot(df2.estimated_weight_g, df2.estimated_weight_g, color = 'red')

In [ ]:
plt.scatter(df2.weight2, df2.weight2 - df2.estimated_weight_g)

import statsmodels.api as sm

X = df2.estimated_weight_g
X = sm.add_constant(X)
Y = df2.weight2 - df2.estimated_weight_g
model = sm.OLS(Y,X)
m = model.fit()
m.summary()


In [ ]:
plt.scatter(df3.estimated_weight_g, df3.weight2)
plt.plot(df3.estimated_weight_g, df3.estimated_weight_g, color = 'red')

In [ ]:
plt.scatter(df4.estimated_weight_g, df4.weight2)
plt.plot(df4.estimated_weight_g, df4.estimated_weight_g, color = 'red')

In [ ]:
count, bins, _ = plt.hist(gt['weight'] * 1000 / 0.83, density = True, color = 'blue', bins = 30)
plt.hist(get_weight(df1.length_adj, df1.width), density = True, alpha = 0.5, color = 'red', bins = bins)

# my_dist2 = pdf_mvsk([np.mean(get_weight(df1.length_adj)), (np.std(get_weight(df1.length_adj)) / 2) ** 2, 0, 1])
# vec2 = np.arange(start=0, stop = 10000, step = .01)

# dist3 = my_dist2(vec2)

# plt.plot(vec2, dist3, lw = 4, color = 'green')


In [ ]:
import scipy.stats as stats
# res = stats.probplot(gt.weight, dist=stats.norm, plot=plt)
res = stats.probplot(gt.weight, dist=stats.lognorm, sparams=(0.15), plot=plt)
# res = stats.probplot(get_weight(df1.length_adj), dist=stats.lognorm, sparams=(0.1), plot=plt)



In [ ]:
plt.hist(df1.akpd_score)

In [ ]:
plt.hist(df1[get_weight(df1.length_adj) > 12000].akpd_score)

In [ ]:
a, b, c = np.mean(gt['weight'] * 1000 / 0.83), np.mean(get_weight(df1.length)), np.mean(get_weight(df1.length_adj))

a, b, c, (a - b) / a, (a - c) / a

In [ ]:
df1['estimated_weight'] = get_weight(df1.lengths)

count, bins, _ = plt.hist(gt['weight'][gt.weight > (5 * 0.83)] * 1000 / 0.83, density = True, color = 'blue', bins = 50)
plt.hist(df1['estimated_weight'][df1['estimated_weight'] > 5000], density = True, alpha = 0.5, color = 'red', bins = bins)

In [ ]:
pen5 = pd.read_csv('blom_vikane_singleweights.csv')

In [ ]:
count, bins, _ = plt.hist(pen5['weight'] * 1000 / 0.83, density = True, color = 'blue', bins = 50)
plt.hist(df2.estimated_weight_g, density = True, alpha = 0.5, color = 'red', bins = bins)

In [ ]:
count, bins, _ = plt.hist(pen5['weight'] * 1000 / 0.83, density = True, color = 'blue', bins = 50)
plt.hist(df2.weight2, density = True, alpha = 0.5, color = 'red', bins = bins)

In [ ]:
plt.scatter(df2.estimated_weight_g, df2.weight2)
plt.plot(df2.estimated_weight_g, df2.estimated_weight_g, color = 'red')

In [ ]:
count, bins, _ = plt.hist(pen5['weight'] * 1000 / 0.83, density = True, color = 'blue', bins = 50)
plt.hist(get_weight(df2.length, df2.width), density = True, alpha = 0.5, color = 'red', bins = bins)

In [ ]:
count, bins, _ = plt.hist(pen5['weight'] * 1000 / 0.83, density = True, color = 'blue', bins = 50)
plt.hist(get_weight(df2.length_adj), density = True, alpha = 0.5, color = 'red', bins = bins)

from statsmodels.sandbox.distributions.extras import pdf_mvsk

my_dist2 = pdf_mvsk([np.mean(get_weight(df2.length_adj)), (np.std(get_weight(df2.length_adj)) / 2) ** 2, 0, 1])
vec2 = np.arange(start=0, stop = 10000, step = .01)

dist3 = my_dist2(vec2)

# plt.plot(vec2, dist3, lw = 4, color = 'green')


In [ ]:
plt.hist(get_weight(df2.length_adj), density = True, alpha = 0.5, color = 'red', bins = bins)

In [ ]:
plt.hist(df1.estimated_k_factor, density = True, alpha = 0.5, color = 'red')
plt.hist(df2.estimated_k_factor, density = True, alpha = 0.5, color = 'blue')

In [ ]:
def get_weight2(length, factor):
    return (length * 23.6068 * factor) ** 3

In [ ]:
count, bins, _ = plt.hist(pen5['weight'] * 1000 / 0.83, density = True, color = 'blue', bins = 50)
plt.hist(get_weight2(df2.length_adj, 0.8) * 2, density = True, alpha = 0.5, color = 'red', bins = bins)

In [ ]:
count, bins, _ = plt.hist(pen5['weight'] * 1000 / 0.83, density = True, color = 'blue', bins = 50)
plt.hist(get_weight3(df2.lengths, df2.estimated_k_factor ** 0.3 / 0.9), density = True, alpha = 0.5, color = 'red', bins = bins)

In [ ]:
a, b, c = np.mean(pen5['weight'] * 1000 / 0.83), np.mean(get_weight(df2.length)), np.mean(get_weight(df2.length_adj))

a, b, c, (a - b) / a, (a - c) / a

In [ ]:
df2['estimated_weight'] = get_weight(df2.lengths)

count, bins, _ = plt.hist(pen5['weight'][pen5.weight > (5 * 0.83)] * 1000 / 0.83, density = True, color = 'blue', bins = 50)
plt.hist(df2['estimated_weight'][df2['estimated_weight'] > 5000], density = True, alpha = 0.5, color = 'red', bins = bins)

In [ ]:
imr_gt = pd.read_csv('imr.csv')

In [ ]:
counts, bins, _ = plt.hist(imr_gt.weight, alpha = 0.5, density = True, color = 'blue', bins = 30)
plt.hist(df4.estimated_weight_g, alpha = 0.5, density = True, color = 'red', bins = bins)

In [ ]:
counts, bins, _ = plt.hist(imr_gt.weight, alpha = 0.5, density = True, color = 'blue', bins = 30)
plt.hist(get_weight(df4.length, df4.width), alpha = 0.5, density = True, color = 'red', bins = bins)